# Model Convergence and Test/Validation Analysis

### The Mission

This is a **very basic** continuous validation test - which acts more as a playground for you to do analysis of train and validation loss. You may test for general acceptability of the validation loss/accuracy or even set thresholds for e.g. underfitting and overfitting.
In this test, we download both binaries for the pickled **train_loss.pkl** and **val_los.pkl** from the Bucket in our Cloud Object Storage instance.

Happy testing! 🥳

In [1]:
import os
import pickle
from ibm_watson_studio_pipelines import WSPipelines

In [ ]:
# Get Cloud API Key from Pipelines Params (Global Parameter)
CLOUD_API_KEY = os.getenv('CLOUD_API_KEY')

In [3]:
## Retrieve cos credentials from pipeline parameters
import json
# Get json from environment and convert to string
project_cos_credentials = json.loads(os.environ['project_cos_credentials'])
mlops_cos_credentials = json.loads(os.getenv('mlops_cos_credentials'))

## PROJECT COS 
AUTH_ENDPOINT = project_cos_credentials['AUTH_ENDPOINT']
#ENDPOINT_URL = project_cos_credentials['ENDPOINT_URL']
#API_KEY_COS = project_cos_credentials['API_KEY']
#BUCKET_PROJECT_COS = project_cos_credentials['BUCKET']

## MLOPS COS
ENDPOINT_URL_MLOPS = mlops_cos_credentials['ENDPOINT_URL']
API_KEY_MLOPS = mlops_cos_credentials['API_KEY']
CRN_MLOPS = mlops_cos_credentials['CRN']
BUCKET_MLOPS  = mlops_cos_credentials['BUCKET']

In [4]:
from ibm_botocore.client import Config
import ibm_boto3

def download_file_cos(local_file_name,key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=API_KEY_MLOPS,
                           ibm_service_instance_id=CRN_MLOPS,
                           ibm_auth_endpoint=AUTH_ENDPOINT,
                           config=Config(signature_version='oauth'),
                           endpoint_url=ENDPOINT_URL_MLOPS)
    try:
        res=cos.download_file(Bucket=BUCKET_MLOPS,Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
        
def load_loss(key, filename):
    download_file_cos(key,filename)
    with open (filename,"rb") as f:
        loss = pickle.load(f)
    return loss

In [5]:
# Download and deserialize pickled dictionary
train_loss = load_loss('train_loss.pkl','train_loss.pkl')
val_loss   = load_loss('val_loss.pkl', 'val_loss.pkl')

train_loss = train_loss['binary_logloss']
val_loss = val_loss['binary_logloss']

File Downloaded


## Test if the model training converges

In [7]:
def check_loss_converge(loss):
    # check every 5 iterations if the loss has decreased
    coverges = [loss[i]>loss[i+4] for i in range(0, len(loss),5)]
    return all(coverges)

In [12]:
training_converges = check_loss_converge(train_loss)
validation_converges = check_loss_converge(val_loss)

## Save the test results to pipeline

In [14]:
params = {}
params['train-loss_converges'] = training_converges
params['val-loss_converges'] = validation_converges

In [15]:
pipelines_client = WSPipelines.from_apikey(apikey=CLOUD_API_KEY)
pipelines_client.store_results(params)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "train_converge": .ibm_watson_studio_pipelines/results/train_converge
